## OHIF Measurement Accuracy Test

# Overview
This Python test script, run from Anaconda/Jupyter tests the accuracy of length measurements created in the OHIF wie..er 
It requires, Windows, Chrome, Brave, Anconda/Jupyter and access to the OHIF viewer.
For more description of prerequsites and context, see the README.txt file in the github repository.

# Data Sources/data used
https://viewer.ohif.org

# Analysis/Model Description
This test is an independent test oracle for OHIF measurements, which calculates the expected a randomly drawn length asum
easuremt in millimeters (mm) based on the pixel spacing of a DICOM image, the scaling factors derived from the image's display size versus its original size, and the deltas (pixel movements) from a simulted mouse movement, using the followng formula:

Expected Measurement Length (mm) = sqrt((Δx * Scaling Factor X * Pixel Spacing X)^2 + (Δy * Scaling Factor Y * Pixel Spacing Y)^2)

Where:
- Δx and Δy are the pixel movements in the X and Y dimensions, respectively.
- Scaling Factor X and Scaling Factor Y are the scaling factors in the X and Y dimensions, respectively, calculated based on the ratio of the DICOM image size to the displayed image size.
- Pixel Spacing X and Pixel Spacing Y are the physical distances in millimeters that each pixel represents in the X and Y dimensions respectively, as obtained from the DICOM header of the image

The expected result is then compared with the actual length as rendered in the browser.
The test passes if all rulers drawn are the same lengths as the expected lengths, within a specified tolerance.

# Author 
Duncan Henderson
(Fulmine Labs LLC)

In [1]:
# Test run variables

# Define a verbose flag (set it to True for verbose output)
#verbose = True
verbose = False

# Set the seed_value (set to None for random seed based on date/time)
seed_value = None
#seed_value = 1703173694627 # Example seed, from log file

# SEt the number of studies to loop over select
study_count = 5

# Set the number of length measurement actions to perform on each study (should be 1 for now)
action_count = 1

# List of studies to pick from (should be studies where we can draw the needed measurements)
study_list = [r'Water Phantom', r'M1', r'CTA Head and Neck']

# Maximum size of the length measurement based on mouse movement (in screen pixels)
max_length_in_screen_pixels = 200

# % Tolerance for difference between expected and actual due to rounding
tolerance_percentage = 1

# Chrome driver location
chrome_driver_location = r'D:\chromedriver-win64\chromedriver.exe'

# Number of times to perform window leveling (may need tuning depending on the window size of the image)
num_window_level_operations = 5

# Distance in pixels to move the mouse when window leveling (may need tuning depending on the screen resolution)
max_distance = 300

# OHIF Viewer location
OHIF_viewer_location = 'https://viewer.ohif.org/'

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By  # Import the By module
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  # Import TimeoutException
from selenium.webdriver.common.keys import Keys
import math
import pygetwindow as gw
import random
import time
import keyboard
import logging
from datetime import datetime
import time
import cv2
import numpy as np
import pytest

In [3]:
# To generate a seed either randomly (e.g., based on date/time) or use a supplied seed_value 
# while also logging the seed used

# Function to generate a seed
def generate_seed(seed_value=None):
    if seed_value is not None:
        seed = seed_value
    else:
        # Generate a seed based on date/time
        seed = int(datetime.now().timestamp() * 1000)  # Use milliseconds for randomness

    logger.iprint ("Selected seed is " + str(seed))
    
    return seed


In [4]:
# Log to a log file that is specific for the test run and also to the screen if verbose is set

class CustomLogger:
    def __init__(self, verbose=False):
        self.verbose = verbose

        # Configure logging
        logging.basicConfig(filename=f'log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log', level=logging.INFO)

    def iprint(self, message):
        if self.verbose:
            print(message)
        logging.info(message)
        
    def eprint(self, message):
        if self.verbose:
            print(message)
        logging.error(message)

In [5]:
# This is the function where the test oracle calculation of expected value is performed based on 
# the data that we already extracted from the measurement, DICOM tags and image
def calculate_expected (pixel_spacing_value, original_dicom_image_size_x, original_dicom_image_size_y, image_display_width, image_display_height, delta_x, delta_y):

    logger.iprint ("In calculate_expected")
    # Assuming pixel_spacing_value is a string like "0.541015625\0.541015625"
    # Split the pixel spacing value into its X and Y components
    spacing_x, spacing_y = map(float, pixel_spacing_value.split("\\"))
  
    # Convert string to integer
    original_dicom_image_size_x = int(original_dicom_image_size_x)  # The number of columns in the DICOM image
    original_dicom_image_size_y = int(original_dicom_image_size_y)  # The number of rows in the DICOM image
    
    # Calculate the scaling factors for width and height
    scaling_factor_x = original_dicom_image_size_x / float(image_display_width)
    scaling_factor_y = original_dicom_image_size_y / float(image_display_height)
    logger.iprint ("scaling_factor_x:" + str(scaling_factor_x))
    logger.iprint  ("scaling_factor_y:" + str(scaling_factor_y))
    
    # Adjust the delta values by the scaling factors
    adjusted_delta_x = delta_x * scaling_factor_x
    adjusted_delta_y = delta_y * scaling_factor_y
    
    # Now use the adjusted deltas to calculate physical distances
    physical_distance_x = adjusted_delta_x * spacing_x
    physical_distance_y = adjusted_delta_y * spacing_y
    
    # Calculate the expected length of the measurement using Pythagoras's theorem
    expected_measurement_length_mm = math.sqrt(physical_distance_x**2 + physical_distance_y**2)
    
    logger.iprint("Expected measurement length in mm: " + str(expected_measurement_length_mm))

    return (expected_measurement_length_mm)


In [6]:
def interact_with_element(driver, locator, action, data=None):
    """
    Interact with a web element identified by the locator.

    Parameters:
    driver (WebDriver): The Selenium WebDriver.
    locator (tuple): Locator tuple, e.g., (By.XPATH, "//div[@data-cy='...']")
    action (str): Action to perform ('click', 'send_keys', 'double_click', etc.).
    data (str, optional): Data to use with the action, like text for send_keys.
    """
    try:
        # Extract the locating mechanism and value from the locator tuple
        by, value = locator

        # Locate the element using the provided locator
        element = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((by, value))
        )

        # Perform the specified action
        if action == 'click':
            element.click()
        elif action == 'send_keys':
            element.clear()
            element.send_keys(data)
        elif action == 'double_click':
            ActionChains(driver).double_click(element).perform()
        # Add more actions as needed
    except TimeoutException:
        logger.eprint(f"Element with locator " + locator + " not found within timeout.")
    except Exception as e:
        logger.eprint(f"Error interacting with element: " + str(e))

In [7]:
# Open a study in the OHIF worklist based on study name, from a list of studies

def select_random_study (study_list):

    logger.iprint ("In select_random_study")

    # Randomly select a study from a list
    study_name = random.choice(study_list )

    logger.iprint (study_name + " selected")

    # Find and click on the a study by its contained text
    interact_with_element(driver, (By.XPATH, f"//span[contains(text(), '{study_name}')]"), 'click')


In [8]:
# The image display size is needed in order to calculate the scaling between DICOM pixels and display pixels
# I have not been able to find a property in the DOM that provides that
# But fear not! We can make the image all white by window levelling it a few times and then detect the size of that
# displayed block of white pixels using OpenCV
# A bit hacky but seems to work ...

def get_image_display_size (driver):

    logger.iprint ("In get_image_display_size")

    # Take a screenshot using Selenium
    driver.save_screenshot('screenshot.png')
    
    # Reload the saved image in color
    original_color = cv2.imread('screenshot.png', cv2.IMREAD_COLOR)
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(original_color, cv2.COLOR_BGR2HSV)
    
    # Define range for white color and apply color thresholding
    lower_white = np.array([0, 0, 168], dtype=np.uint8)
    upper_white = np.array([172, 111, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_white, upper_white)
    
    # Find contours on the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by size
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]
    
    # Display the result to check the contours are identifying the image correctly
    if (verbose):
        # Draw the filtered contours on the original color image with a red outline
        cv2.drawContours(original_color, filtered_contours, -1, (0, 0, 255), 2)
        cv2.imshow('Filtered Contours on Original', original_color)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    # Assuming the largest contour corresponds to the image
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    logger.iprint(f"Displayed Image Width: " + str(w) + "px")
    logger.iprint(f"Displayed Image Height: " + str(h) + "px")

    return (w, h)


In [9]:
# Deal with the 'Track measurements for series?' dialog
def track_measurements_for_series (driver):

    logger.iprint ("In track_measurements_for_series")
    
    # Check if the message "Track measurements for this series?" is present
    try:
        message_element = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'flex grow items-center')]//span[text()='Track measurements for this series?']"))
        )

        # If the message is present, try to locate and click the "Yes" button
        try:
            interact_with_element(driver, (By.XPATH, "//button[@data-cy='prompt-begin-tracking-yes-btn']"), 'click')
            logger.iprint("Track measurements message found and 'Yes' button clicked")
        except TimeoutException:
            logger.iprint("Yes button not found or not clickable")
    except TimeoutException:
        if verbose:
                logger.eprint("Track measurements message not found or not clickable")

In [10]:
# Function to simulate drawing a length measurement
def image_mouse_movement(driver, max_distance, selected_tool, random_mode='random'):

    logger.iprint ("In image_mouse_movement")
    
    try:
    
        # Locate the image element by class name
        image_element = driver.find_element(By.CLASS_NAME, "cornerstone-viewport-element")

        # Move to the image element
        ActionChains(driver).move_to_element(image_element).perform()

        if (random_mode == 'random'):
        
            # Limit the random values for move_distance_x and move_distance_y
            move_distance_x = random.randint(-1 * max_distance, max_distance)  # Adjust the range as needed
            move_distance_y = random.randint(-1 * max_distance, max_distance)  # Adjust the range as needed

        else:
            # We are trying to brighten the image, move to a fixed position
            move_distance_x = max_distance
            move_distance_y = max_distance

        time.sleep(0.1)
        ActionChains(driver).move_by_offset(move_distance_x, move_distance_y).click_and_hold().perform()

        if (random_mode == 'random'):
            
            # Limit the random values for move_distance_x and move_distance_y
            move_distance_x2 = random.randint(-1 * max_distance, max_distance)  # Adjust the range as needed
            move_distance_y2 = random.randint(-1 * max_distance, max_distance)  # Adjust the range as needed

        else:
            # We are trying to brighten the image, move a maximum distance up (in the y plane only)
            move_distance_x2 = 0
            move_distance_y2 = (max_distance * -1) -1


        time.sleep(0.1)
        ActionChains(driver).move_by_offset(move_distance_x2, move_distance_y2).click_and_hold().perform()

        # Release the left mouse button
        time.sleep(0.1)
            
        ActionChains(driver).release().perform()

        # Calculate the differences in x and y coordinates
        delta_x = move_distance_x2
        delta_y = move_distance_y2
        
        # Calculate the length of the measurement using Pythagoras's theorem
        measurement_length_pixels = math.sqrt(delta_x**2 + delta_y**2)
        
        logger.iprint("Measurement length in pixels: " + str(measurement_length_pixels))
                
    except Exception as e:
            logger.eprint("Error during mouse movement:")  

    return (delta_x, delta_y)                


In [11]:
# Function to extract the measurement value from the DOM using a Javascript call from Selenium
def extract_the_measurement_value (driver):

    logger.iprint ("In extract_the_measurement_value")

    try:
        # JavaScript script to extract the measurement value
        js_script = """
        var svgTextElements = document.querySelectorAll('svg.svg-layer g text tspan');
        for (var i = 0; i < svgTextElements.length; i++) {
            if (svgTextElements[i].textContent.includes('mm')) {
                return svgTextElements[i].textContent;
            }
        }
        return null;
        """
        actual_measurement_length = driver.execute_script(js_script)
    
        if actual_measurement_length:
            logger.iprint("Length Measurement:" + actual_measurement_length)
        else:
            logger.iprint("Measurement value not found or does not contain 'mm'.")
    
    except Exception as e:
        logger.iprint("Error executing JavaScript:" + str(e))

    return (actual_measurement_length)


In [12]:
# Get a DICOM tag value from the Tag Browser

def get_dicom_tag_value(driver, tag_name):

    logger.iprint ("In get_dicom_tag_value")
    
    interact_with_element(driver, (By.CSS_SELECTOR, "input.border-inputfield-main"), 'send_keys', tag_name)

    try:
        # Wait for the element that contains the tag value to be visible
        wait = WebDriverWait(driver, 3)  # Timeout after 3 seconds
        parent_element_xpath = f"//div[contains(text(), '{tag_name}')]/following-sibling::div"
        tag_value_element = wait.until(EC.visibility_of_element_located((By.XPATH, parent_element_xpath)))

        # Extract and return the tag value
        logger.iprint ("tag_value_element text is " + tag_value_element.text)      
        tag_text = tag_value_element.text
            
    except TimeoutException:
        logger.eprint(f"Timed out waiting for the " + tag_name + " value element to appear.")
        return None

    # Send the ESC key to the body of the page
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()

    logger.iprint("Sent ESC key to close the Tag Browser window.")

    return (tag_text) 



In [13]:
# Function to simulate drawing a Window Level operation

def perform_window_level(driver, window_level_movement):

    logger.iprint ("perform_randomized_window_level selected")

    interact_with_element(driver, (By.XPATH, "//button[@data-cy='WindowLevel-split-button-primary']"), 'click', data=None)

    image_mouse_movement(driver, window_level_movement, "window_level", "non_random")    

In [14]:
# Function to select a randomized measurement tool

def select_randomized_more_measure_tools(driver):

    logger.iprint("select_randomized_more_measure_tools selected")
    
    # Starting with length, can add other measurements following the same methodology
    # selectable_tools = ["Length", "ArrowAnnotate", "Bidirectional", "EllipticalROI", "CircleROI"]
    selectable_tools = ["Length"]

    # Select a random tool data-cy value
    selected_tool = random.choice(selectable_tools)

    logger.iprint ("Selected tool is: " + selected_tool)

    # Set the wait time to 2 seconds
    wait = WebDriverWait(driver, 2)
    tool_button_selector = f'button[data-tool="{selected_tool}"]'
    
    try:
        # Try to locate the tool button
        tool_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, tool_button_selector)))
        tool_button.click()  # Click the button if found
    except TimeoutException:
        # Handle the case where the element is not found within the wait time
        logger.iprint (f"Tool '{selected_tool}' button not found within 2 seconds, trying dropdown")
            
        try:
            # Locate the "More Measure Tools" dropdown element by its data-cy attribute
            interact_with_element(driver, (By.XPATH, "//div[@data-cy='MeasurementTools-split-button-secondary']"), 'click')
       
            # Locate the tool element by its data-cy attribute
            interact_with_element(driver, (By.XPATH, f"//div[@data-cy='{selected_tool}']"), 'click')
      
        except Exception as e:
            if (verbose):
                logger.eprint(f"Error while selecting a control in more measure tools: {str(e)}")

    physical_distance_x, physical_distance_y = image_mouse_movement(driver, max_length_in_screen_pixels, selected_tool)
             
    track_measurements_for_series (driver)
    
    actual_length_measurement = extract_the_measurement_value (driver)

    return (physical_distance_x, physical_distance_y, actual_length_measurement)


In [15]:
# Function to select a randomized more tools control

def select_tag_browser (driver):

    logger.iprint("select_tag_browser")
    
    selected_tool = "TagBrowser"

    # Set the wait time to 2 seconds
    wait = WebDriverWait(driver, 2)
    tool_button_selector = f'button[data-tool="{selected_tool}"]'
    
    try:
        # Try to locate the tool button
        tool_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, tool_button_selector)))
        tool_button.click()  # Click the button if found
    except TimeoutException:
        # Handle the case where the element is not found within the wait time
        logger.iprint (f"Tool '{selected_tool}' button not found within 2 seconds, trying dropdown.")
          
        try:
            # Locate the "More Measure Tools" dropdown element by its data-cy attribute
            interact_with_element(driver, (By.XPATH, "//div[@data-cy='MoreTools-split-button-secondary']"), 'click')
            
            interact_with_element(driver, (By.XPATH, f"//div[@data-cy='{selected_tool}']"), 'click')
       
        except Exception as e:
            logger.eprint(f"Error while selecting the more tools control: " + dropdown_element.text)


In [16]:
# Main code execution
logger = CustomLogger(verbose)

In [17]:
# Generate the seed and record it
seed = generate_seed(seed_value)

# Set the seed for random number generation
random.seed(seed)

In [18]:

# Set the path to the ChromeDriver executable as an environment variable
os.environ['webdriver.chrome.driver'] = chrome_driver_location

# Initialize the Chrome web driver
driver = webdriver.Chrome()

# Create ChromeOptions object to configure the driver
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = False  # Set headless mode to False

# Get the Chrome window by its title
chrome_window = gw.getWindowsWithTitle('Google Chrome')[0]  # Adjust the title if needed

# Maximize the Chrome window
time.sleep(2)
chrome_window.maximize()

# Initialize a variable to store the previously selected tool
previous_measure_tool = None
previous_more_tool = None

In [19]:
# Find the Brave window by its title
brave_window = gw.getWindowsWithTitle("Brave")

if brave_window:
    # Minimize the Brave window
    brave_window[0].minimize()
else:
    logger.eprint("Brave window not found")

In [20]:
# Get the current active window
current_window = gw.getActiveWindow()

# Minimize the current window
current_window.minimize()


In [21]:
# Random study selection
for study in range(study_count):  

    # (Re)open OHIF Viewer
    driver.get(OHIF_viewer_location)

    # Wait for the OHIF Viewer to load
    time.sleep(2)
    
    select_random_study (study_list)

    # Open Basic Viewer
    
    interact_with_element(driver, (By.XPATH, "//button[contains(., 'Basic Viewer')]"), 'double_click')
    
    time.sleep(1)

    # Window level the image several times to make it visible in the viewframe
    
    # Perform window level operation multiple times
    for _ in range(num_window_level_operations):
        perform_window_level(driver, max_distance)

    # Now you can calculate the scale factor based on these dimensions
    image_display_width, image_display_height = get_image_display_size (driver)
    
    # Random operations on the study
    for operation in range(action_count):  # Repeat random actions with random parameters
        logger.iprint ("In loop: study " + str(study) + " action " + str(operation))
    
        # Randomly select an action
        selected_action = random.choice([select_randomized_more_measure_tools])
          
        # Call the selected action
        delta_x, delta_y, actual_measurement_length = selected_action(driver)
        logger.iprint ("delta_x: " + str(delta_x))
        logger.iprint ("delta_y: " + str(delta_y))

        # Select the Tag Browser and extract the needed tag values
        select_tag_browser(driver)
        pixel_spacing_value = get_dicom_tag_value(driver, "PixelSpacing")
        select_tag_browser(driver)
        original_dicom_image_size_x = get_dicom_tag_value(driver, "Columns")
        select_tag_browser(driver)
        original_dicom_image_size_y = get_dicom_tag_value(driver, "Rows")
        
        logger.iprint ("Pixel Spacing Value:" + pixel_spacing_value)
        logger.iprint ("original_dicom_image_size_x:" + original_dicom_image_size_x)
        logger.iprint ("original_dicom_image_size_y:" + original_dicom_image_size_y)

        # Use the data that we have collected to calculate the expected value for the length
        expected_measurement_length_mm = calculate_expected (pixel_spacing_value, original_dicom_image_size_x, original_dicom_image_size_y, image_display_width, image_display_height, delta_x, delta_y)
        
        logger.iprint("Expected measurement length in mm: " + str(expected_measurement_length_mm))
        logger.iprint("Actual measurement length: " + str(actual_measurement_length))
        
        # Parse the numerical part of the actual measurement length string
        actual_length_value = float(actual_measurement_length.split()[0])
        
        # Set a tolerance for the comparison (going with 1% of expected for illustration)
        tolerance = expected_measurement_length_mm * (tolerance_percentage * 0.01)
        
        # Calculate the absolute difference between expected and actual lengths
        # As the actual seems to be rounded to the nearest two decimal places, we can do the same
        difference = abs(round(expected_measurement_length_mm, 3) - actual_length_value)
        logger.iprint("The difference between expected and actual is: " + str(difference))
        
        assert difference <= tolerance, \
               f"Expected length {expected_measurement_length_mm} mm, but got {actual_length_value}"
        logger.iprint("Assertion passed: The actual length is within the expected tolerance of " + str(tolerance_percentage) + "%.")

